[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nils-holmberg/socs-qmd/blob/main/jnb/lab4_cv1.ipynb)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# image urls

In [ ]:
#!pip install gdown
!gdown https://drive.google.com/uc?id=1EMzJxxoBaN_NbvF7xhoc09K82vQ6H_LX

In [ ]:
fp = "content.xlsx"
df = pd.read_excel(fp, header=None, names=['id', 'image', 'text'])
df.head()

In [ ]:
import requests
import os

# Directory to save images
image_dir = 'ibs'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

# Function to download an image
def download_image(url, folder):
    response = requests.get(url)
    if response.status_code == 200:
        # Extract image file name
        filename = url.split('/')[-1]
        file_path = os.path.join(folder, filename)
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download {url}")

# Download images from URLs in the DataFrame
for url in df['image']:
    download_image(url, image_dir)


# image archive

In [ ]:
!gdown https://drive.google.com/uc?id=1sBsckDIyQJ-zTsEpjCntYMIJcbBpF3wk

In [ ]:
!unzip content-images.zip

# display images

In [ ]:
#!pip install -q opencv-python
import cv2
from google.colab.patches import cv2_imshow

# Load an image from file
image_path = 'images/ibs-92.jpg'
image = cv2.imread(image_path)

# Check if the image was successfully loaded
if image is not None:
    # Display the image
    cv2_imshow(image)
else:
    print("Error: Unable to load image.")


In [ ]:
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

# Read the image using OpenCV
image_path = 'images/ibs-92.jpg'  # Replace with your image path
image = cv2.imread(image_path)

# Convert the color scheme from BGR to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Use Seaborn to improve plot aesthetics
sns.set()  # Apply Seaborn styles

# Display the image
plt.imshow(image_rgb)
plt.axis('off')  # Hide the axis
plt.show()


# describe images

In [ ]:
import os
import cv2
import pandas as pd

# Path to the directory containing images
directory = 'images'  # Replace with your directory path

# Initialize a list to store image data
image_data = []

# Iterate through each file in the directory
for filename in os.listdir(directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Add other image formats if needed
        # Read the image
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path)

        # Extract image dimensions and channels
        height, width, channels = img.shape

        # Append the data to the list
        image_data.append({'Filename': filename, 'Width': width, 'Height': height, 'Channels': channels})

# Create a DataFrame from the image data
image_df = pd.DataFrame(image_data)

# Display the DataFrame
image_df.head()


# image to grayscale

In [ ]:
import numpy as np
import cv2
import seaborn as sns
import matplotlib.pyplot as plt

# For demonstration, create a sample color image using NumPy
#color_image = np.random.randint(0, 256, (200, 200, 3), dtype=np.uint8)

# Read the image using OpenCV
image_path = 'images/ibs-92.jpg'  # Replace with your image path
color_image = cv2.imread(image_path)

# Convert the image to grayscale
gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

# Increase intensities by 25% with a ceiling of 255
enhanced_gray_image = np.clip(gray_image * 0.01, 0, 255).astype(np.uint8)

# Display the modified gray image using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(enhanced_gray_image, annot=False, cmap='gray')
plt.axis('off')  # Hide the axis
plt.title('Altered Grayscale Image')
plt.show()


# images from video

In [ ]:
#https://github.com/kannagi0303/yt-dlp-gui
#https://github.com/section83/MacYTDL
#https://drive.google.com/file/d/1T9S32UwmDnUd6YbxRYfyG7-gud8jh2Y_/view?usp=sharing
!gdown https://drive.google.com/uc?id=1T9S32UwmDnUd6YbxRYfyG7-gud8jh2Y_


In [ ]:
import cv2
import os

# Path to the video file
video_path = 'content-video.mp4'

# Create a VideoCapture object
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Directory to save frames
save_dir = 'images-video'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

frame_idx = 0
while True:
    ret, frame = cap.read()

    # Break the loop if there are no more frames
    if not ret:
        break

    # Save every 100th frame
    if frame_idx % 100 == 0:
        # Filename with 5 leading zeroes
        save_path = os.path.join(save_dir, f'frame_{frame_idx:05d}.jpg')
        cv2.imwrite(save_path, frame)
        print(f'Saved frame {frame_idx:05d}')

    frame_idx += 1

# Release the VideoCapture object
cap.release()


# normalize images, preprocessing

In [ ]:
import cv2
import os
import numpy as np

def resize_and_crop(img, size):
    # Resize image to maintain aspect ratio
    h, w, _ = img.shape
    if h > w:
        new_h, new_w = size * h / w, size
    else:
        new_h, new_w = size, size * w / h

    new_h, new_w = int(new_h), int(new_w)
    resized_img = cv2.resize(img, (new_w, new_h))

    # Crop the center of the image
    startx = new_w//2 - size//2
    starty = new_h//2 - size//2
    return resized_img[starty:starty+size, startx:startx+size]

def normalize_image(img):
    # Normalize pixel values to [0, 1]
    return img / 255.0

directory = 'images-video'
output_directory = 'images-normalize'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for filename in os.listdir(directory):
    if filename.endswith('.jpg'):
        img = cv2.imread(os.path.join(directory, filename))
        img = resize_and_crop(img, 256)
        normalized_img = normalize_image(img)

        # Convert the normalized image back to 8-bit format
        img_to_save = (normalized_img * 255).astype(np.uint8)

        # Save the normalized image
        output_path = os.path.join(output_directory, filename)
        cv2.imwrite(output_path, img_to_save)


In [ ]:
# Read the image using OpenCV
image_path = 'images-normalize/frame_00000.jpg'  # Replace with your image path
image = cv2.imread(image_path)

# Convert the color scheme from BGR to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Use Seaborn to improve plot aesthetics
sns.set()  # Apply Seaborn styles

# Display the image
plt.imshow(image_rgb)
plt.axis('off')  # Hide the axis
plt.show()

In [ ]:
#!rm images-normalize/*

# manipulate regions

In [ ]:
import numpy as np
import cv2
import seaborn as sns
import matplotlib.pyplot as plt

# Load your RGB image
image_path = 'images-normalize/ibs-92.jpg'  # Replace with your image path
image = cv2.imread(image_path)

# Ensure the image was loaded
if image is None:
    print("Error: Image not found.")
    exit()

# OpenCV reads in BGR format, so convert it to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Dimensions for the top left quadrant
height, width, _ = image.shape
top_left_height = height // 2
top_left_width = width // 2

# Access the green channel (channel 1) in the top left quadrant
green_channel = image[:top_left_height, :top_left_width, 1]

# Replace intensities over 50 with 255 in the green channel of the top left quadrant
green_channel[green_channel > 50] = 255

# Update the green channel in the original image's top left quadrant
image[:top_left_height, :top_left_width, 1] = green_channel

# Display the modified image using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(image[:, :, 1], annot=False, cmap='Greens')
plt.axis('off')  # Hide the axis
plt.title('Modified Image with Adjusted Green Channel in Top Left Quadrant')
plt.show()



In [ ]:
import numpy as np
import cv2
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming the image is already loaded as a NumPy array in variable 'image'
# For demonstration, let's create a sample RGB image
#image = np.random.randint(0, 256, (200, 200, 3), dtype=np.uint8)

# Load your RGB image
image_path = 'images-normalize/ibs-92.jpg'  # Replace with your image path
image = cv2.imread(image_path)

# Ensure the image was loaded
if image is None:
    print("Error: Image not found.")
    exit()

# OpenCV reads in BGR format, so convert it to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Dimensions for the top left quadrant
height, width, _ = image.shape
top_left_height = height // 2
top_left_width = width // 2

# Access the green channel (channel 1) in the top left quadrant
green_channel = image[:top_left_height, :top_left_width, 1]

# Replace intensities over 50 with 255 in the green channel of the top left quadrant
green_channel[green_channel > 150] = 255

# Update the green channel in the original image's top left quadrant
image[:top_left_height, :top_left_width, 1] = green_channel

# Display the original image (with the modified green channel) without heatmap visualization or color maps
plt.figure(figsize=(8, 6))
plt.imshow(image)
plt.axis('off')  # Hide the axis
plt.title('Original Image with Altered Green Channel Intensities')
plt.show()



# image similarity

In [ ]:
import os
from skimage import io, img_as_float
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import seaborn as sns

# Function to calculate SSIM between consecutive images
def calculate_ssim(images):
    ssim_values = []
    for i in range(len(images) - 1):
        image1 = img_as_float(images[i])
        image2 = img_as_float(images[i + 1])
        ssim_value = ssim(image1, image2, multichannel=True)
        ssim_values.append(ssim_value)
    return ssim_values

# Directory containing images
image_directory = 'images-normalize'  # Replace with your directory path

# Create a list of image file paths
image_paths = [os.path.join(image_directory, file) for file in os.listdir(image_directory) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Load images from the file paths
images = [io.imread(path) for path in image_paths]

# Calculate SSIM for each pair of consecutive images
ssim_values = calculate_ssim(images)

# Plotting the similarity as a function of image number
plt.figure(figsize=(10, 6))
sns.lineplot(x=range(1, len(ssim_values) + 1), y=ssim_values)
plt.xlabel('Image Number')
plt.ylabel('SSIM Value')
plt.title('Image Similarity as a Function of Image Number')
plt.show()


In [ ]:
from skimage.metrics import structural_similarity as compare_images
import cv2

# Assuming we have a path to two images, for demonstration let's create two sample images
#image1 = (np.random.rand(100, 100, 3) * 255).astype(np.uint8)
#image2 = (np.random.rand(100, 100, 3) * 255).astype(np.uint8)
image1 = cv2.imread('images-normalize/frame_00000.jpg')
image2 = cv2.imread('images-normalize/frame_00100.jpg')

# Convert images to grayscale for SSIM computation
gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Compute SSIM between the two images, also returning the difference image
ssim, difference_image = compare_images(gray_image1, gray_image2, full=True)
difference_image = (difference_image * 255).astype(np.uint8)

# Plotting the images side by side using seaborn
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Display first image
sns.heatmap(gray_image1, ax=axes[0], cbar=False, cmap='gray')
axes[0].set_title('Image 1')
axes[0].axis('off')

# Display second image
sns.heatmap(gray_image2, ax=axes[1], cbar=False, cmap='gray')
axes[1].set_title('Image 2')
axes[1].axis('off')

# Display difference image
sns.heatmap(difference_image, ax=axes[2], cbar=False, cmap='gray')
axes[2].set_title('Difference Image')
axes[2].axis('off')

plt.show()



# image features, sift

Scale-Invariant Feature Transform (size and orientation)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

# Read an image using OpenCV
image_path = 'images-normalize/frame_00000.jpg'  # Replace with your image path
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
# Read an image using OpenCV (For demonstration, using a sample image)
#image = np.random.randint(0, 256, (128, 128), dtype=np.uint8)

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect SIFT features
keypoints, _ = sift.detectAndCompute(image, None)

# Draw keypoints on the image with their size and orientation
sift_image = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Plotting the original image and SIFT features using seaborn
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Display the original image
sns.heatmap(image, ax=axes[0], cbar=False, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

# Display the image with SIFT features
axes[1].imshow(sift_image, cmap='gray')
axes[1].set_title('SIFT Features')
axes[1].axis('off')

plt.show()


In [ ]:
import pandas as pd

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect SIFT features
keypoints, descriptors = sift.detectAndCompute(image, None)

# Extract information from keypoints and store in a pandas DataFrame
sift_features = []
for kp in keypoints:
    sift_features.append({'x': kp.pt[0], 'y': kp.pt[1], 'angle': kp.angle,
                          'size': kp.size, 'response': kp.response,
                          'octave': kp.octave, 'class_id': kp.class_id})

# Create a DataFrame from the SIFT features
df_sift_features = pd.DataFrame(sift_features)

# Display the DataFrame
df_sift_features



# construct images

In [ ]:
import numpy as np
import cv2
import seaborn as sns
import matplotlib.pyplot as plt

# Create an empty (black) 256x256 image
empty_image = np.zeros((256, 256, 3), dtype=np.uint8)

# Display the image using seaborn
sns.set()  # For a nicer plot style
plt.imshow(cv2.cvtColor(empty_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for correct color representation
plt.axis('off')  # Hide axis
plt.show()

# manipulate color channels

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Function to create an image with specified RGB values
def create_rgb_image(red, green, blue):
    image = np.zeros((256, 256, 3), dtype=np.uint8)
    image[:, :, 0] = red    # Red channel
    image[:, :, 1] = green  # Green channel
    image[:, :, 2] = blue   # Blue channel
    return image

# Creating images for each combination
combinations = [
    (0, 0, 0),    # Black
    (255, 0, 0),  # Red
    (0, 255, 0),  # Green
    (0, 0, 255),  # Blue
    (255, 255, 0),# Yellow
    (255, 0, 255),# Magenta
    (0, 255, 255),# Cyan
    (255, 255, 255)# White
]

images = [create_rgb_image(*combo) for combo in combinations]

# Plotting
sns.set()  # For a nicer plot style
plt.figure(figsize=(12, 6))

for i, img in enumerate(images):
    plt.subplot(2, 4, i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'R:{combinations[i][0]} G:{combinations[i][1]} B:{combinations[i][2]}')

plt.tight_layout()
plt.show()


# manipulate shapes

In [ ]:
import numpy as np
import cv2
import seaborn as sns
import matplotlib.pyplot as plt

# Start with a black image
image = np.zeros((256, 256, 3), dtype=np.uint8)

# List to store each step
images = [image.copy()]

# Add a red line
image_with_line = image.copy()
cv2.line(image_with_line, (50, 50), (200, 50), (255, 0, 0), 3)  # Red line
images.append(image_with_line)

# Add a green rectangle
image_with_rectangle = image_with_line.copy()
cv2.rectangle(image_with_rectangle, (60, 80), (180, 150), (0, 255, 0), 3)  # Green rectangle
images.append(image_with_rectangle)

# Add a blue oval
image_with_oval = image_with_rectangle.copy()
cv2.ellipse(image_with_oval, (120, 200), (60, 30), 0, 0, 360, (0, 0, 255), -1)  # Blue oval
images.append(image_with_oval)

# Plotting
sns.set()  # For a nicer plot style
plt.figure(figsize=(12, 6))

shape_titles = ["Original", "With Line", "With Rectangle", "With Oval"]
for i, img in enumerate(images):
    plt.subplot(1, len(images), i + 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
    plt.axis('off')
    plt.title(shape_titles[i])

plt.tight_layout()
plt.show()


# manipulate pixel regions

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Create an initial black image
black_image = np.zeros((256, 256, 3), dtype=np.uint8)

# Function to add a colored square to an existing image at a specified position
def add_square_to_image(image, color, position, square_size=50):
    x_start, y_start = position
    image_with_square = image.copy()
    image_with_square[y_start:y_start + square_size, x_start:x_start + square_size] = color
    return image_with_square

# Add a red square in the top left corner
red = [255, 0, 0]
image_red_square = add_square_to_image(black_image, red, (0, 0))

# Plotting both images side by side
sns.set()  # For a nicer plot style
plt.figure(figsize=(10, 5))

# Plot original black image
plt.subplot(1, 2, 1)
plt.imshow(black_image)
plt.axis('off')
plt.title("Original Black Image")

# Plot image with red square
plt.subplot(1, 2, 2)
plt.imshow(image_red_square)
plt.axis('off')
plt.title("Image with Red Square")

plt.tight_layout()
plt.show()


# image features

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read the image
image = cv2.imread('images/ibs-92.jpg')

# Calculate color histograms for each channel
hist_r = cv2.calcHist([image], [0], None, [256], [0, 256])
hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
hist_b = cv2.calcHist([image], [2], None, [256], [0, 256])

# Normalize histograms
hist_r /= hist_r.sum()
hist_g /= hist_g.sum()
hist_b /= hist_b.sum()

# Create subplots for the original image and histograms
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot the original image
axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image')
axes[0].axis('off')

# Plot the color histograms using Seaborn
sns.lineplot(x=np.arange(256), y=hist_r.squeeze(), color='red', ax=axes[1], label='Red')
sns.lineplot(x=np.arange(256), y=hist_g.squeeze(), color='green', ax=axes[1], label='Green')
sns.lineplot(x=np.arange(256), y=hist_b.squeeze(), color='blue', ax=axes[1], label='Blue')
axes[1].set_title('Color Histograms')
axes[1].set_xlabel('Pixel Value')
axes[1].set_ylabel('Frequency')
axes[1].legend()

# Show the plots
plt.tight_layout()
plt.show()


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Read the image
image = cv2.imread('images/ibs-92.jpg', cv2.IMREAD_GRAYSCALE)

# Define edge detection settings
edge_detection_settings = [
    ('Canny', cv2.Canny(image, 50, 150)),
#    ('Laplacian', cv2.Laplacian(image, cv2.CV_64F)),
    ('Sobel_X', cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)),
    ('Sobel_Y', cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5))
]

# Iterate through edge detection settings
for setting_name, edges in edge_detection_settings:
    # Convert the edges to binary (keeping them in 0-255 range)
    edges_binary = np.where(edges > 0, 255, 0).astype(np.uint8)

    # Count the number of white edge pixels
    white_pixel_count = cv2.countNonZero(edges_binary)

    # Concatenate the original image and binary edge image horizontally
    concatenated_image = np.hstack((image, edges_binary))

    # Display the concatenated image using cv2_imshow
    print(f'Edge Detection Algorithm: {setting_name}')
    cv2_imshow(concatenated_image)
    print(f'Number of white edge pixels: {white_pixel_count}\n')

    # Save the concatenated image with the specified filename prefix
    filename = f'edges_{setting_name}.jpg'
    cv2.imwrite(filename, concatenated_image)

    print(f'Saved as {filename}\n')


# kernel images and convolutions

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Horizontal Sobel Kernel (detects vertical edges)
sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]])

# Vertical Sobel Kernel (detects horizontal edges)
sobel_y = np.array([[-1, -2, -1],
                    [ 0,  0,  0],
                    [ 1,  2,  1]])

# Plot the kernels
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
sns.heatmap(sobel_x, annot=True, cmap='coolwarm', ax=ax[0], cbar=False)
ax[0].set_title('Horizontal Sobel Kernel')
sns.heatmap(sobel_y, annot=True, cmap='coolwarm', ax=ax[1], cbar=False)
ax[1].set_title('Vertical Sobel Kernel')
plt.show()
